# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 \|\theta\|^2 + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

In [231]:
import numpy as np
import yaml

In [232]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])

theta0 = np.array([1.0, 1.0, 0.0])

lr = 0.1

def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5



$\frac{\mathrm d}{\mathrm d \theta_k}\mathcal{L}(\theta) = 0.2*\theta_k + \frac{1}{N}\sum\limits_{i=1}^N \begin{cases}
0 & \text{if  }  (1 - y_i f(x_i, \theta)) \leq 0 \\
-y_i * \frac{\mathrm d}{\mathrm d \theta_k}f(x_i, \theta)& \text{otherwise }
\end{cases}
$

In [233]:
# Производная f по theta
def f_diff(X):
  return np.array([X[:, 0], X[:, 1], [1] * len(X)])

# Производная loss по theta
def loss_diff(X, y, theta):
  theta = np.asarray(theta)
  norm_diff = 2 * theta

  # Маска для расчета производной функции maximum
  deltas = y * f(X, theta)
  deltas = np.maximum(0, 1 - deltas)
  mask  = (deltas != 0) # аналогично mask = [0 if x == 0 else 1 for x in deltas]
  mask = mask.astype(np.int64) # deltas = [0 0 0 1 0.5], mask  = [0 0 0 1 1]

  deltas_diff = mask * (-1 * y * f_diff(X))
  return 0.1 * norm_diff + np.mean(deltas_diff, axis = 1)

# шаг
def step(X, y, theta, lr):
  return theta - lr * loss_diff(X, y, theta)

In [234]:
theta = theta0

for _ in range(1000):
  theta = step(X, y, theta, lr)

In [235]:
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Prediction: [ 0.75218494  2.00436988  1.00436988 -0.37390753 -0.74781506]
Loss: 0.48815643632236133
